# Agent Executor

## Setup

In [1]:
!pip install --quiet -U langgraph langchain langchain_openai langchainhub tavily-python

In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

## LangChain Agent 만들기

먼저, LangChain 에이전트를 생성하겠습니다. LangChain 에이전트에 대한 자세한 정보는 [이 문서](https://python.langchain.com/v0.2/docs/concepts/#agents)를 참조하세요.

- legacy agents: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/

In [3]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai.chat_models import ChatOpenAI

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o", streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

## Graph State 정의하기

이제 그래프 상태를 정의하겠습니다. 전통적인 LangChain 에이전트의 상태에는 몇 가지 속성이 있습니다:

1. `input`: 이는 사용자로부터 전달된 주요 질문을 나타내는 입력 문자열입니다.
2. `chat_history`: 이는 이전 대화 메시지로, 입력으로 전달됩니다.
3. `intermediate_steps`: 이는 에이전트가 시간에 따라 수행한 액션과 이에 대한 관찰 목록입니다. 에이전트의 각 반복마다 업데이트됩니다.
4. `agent_outcome`: 이는 에이전트의 응답으로, AgentAction 또는 AgentFinish가 될 수 있습니다. 에이전트가 AgentFinish일 경우, AgentExecutor는 작업을 완료해야 하며, 그렇지 않으면 요청된 도구를 호출해야 합니다.


In [4]:
import operator
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## 노드 정의

이제 그래프에서 몇 가지 다른 노드를 정의해야 합니다. `langgraph`에서 노드는 함수 또는 [Runnable](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel)일 수 있습니다. 이를 위해 필요한 주요 노드는 다음과 같습니다:

1. Agent: 어떤 액션을 취할지 결정하는 역할을 담당합니다.
2. 도구를 호출하는 함수: 에이전트가 액션을 취하기로 결정한 경우, 이 노드가 해당 액션을 실행합니다.

In [5]:
from langchain_core.agents import AgentFinish

from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)


# Define the agent
def run_agent(state):
    agent_outcome = agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}


# Define the function to execute tools
def execute_tools(state):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = state["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


# Define logic that will be used to determine which conditional edge to go down
def should_continue(state):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(state["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

## Graph 정의하기

In [6]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [7]:
inputs = {"input": "미국 대선 언제야? 검색해서 알려줘", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': '미국 대선 날짜 2024'}, log="\nInvoking: `tavily_search_results_json` with `{'query': '미국 대선 날짜 2024'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "미국 대선 날짜 2024"\n}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-9c1872f7-a99a-45de-8918-5d7fa5f0ae9c-0')])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': '미국 대선 날짜 2024'}, log="\nInvoking: `tavily_search_results_json` with `{'query': '미국 대선 날짜 2024'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "미국 대선 날짜 2024"\n}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-9c1872f7-a99a-45de-8918-5d7fa5f0ae9c-0')]), "[{'url': 'https://namu.wiki/w/2024년 미국 대통령 선거', '